In [1]:
import collections
import math
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
#  编码器

class Seq2SeqEncoder(d2l.Encoder):
    # 用于序列到序列学习的循环神经网络编码器

    def __init__(self,vocab_size,embed_size,num_hiddens,num_layers,dropout=0,**kwargs):
        super(Seq2SeqEncoder,self).__init__(**kwargs)

        #  嵌入
        # 将每一个单词转换为embed_szie个向量 单词数量是vocab_size
        # 将一个单词转换成 50个维度的向量
        self.embedding  = nn.Embedding(vocab_size,embed_size)

        # RNN层  将embed_size 转换成 num_hiddens  比如50 -> 20 个特征
        # num_layers是RNN的层
        self.rnn = nn.GRU(embed_size,num_hiddens,num_layers,dropout=dropout)


    def forward(self, X, *args):
        # 前向传播  X的形状是  batch_size num_steps, embed_size
        X = self.embedding(X)

        # 循环神经网络中 第一个轴对应于时间步 也就是多少个单词数量
        # 举一个例子 比如X = torch.randn(10,3,100)
        # 那么10就是代表 num_steps 也就是时间步  其实就是每一个句子的单词数量
        # 3 代表有三个句子  100 代表每一个单词是100个特征进行表示
        X = X.permute(1,0,2) # num_steps参数放在前面

        # RNN正向传播  这里只传入向量  返回所有时间步的最后一层的隐藏状态
        # state是最后一个时间步的所有层隐藏状态
        # output的形状  num_steps,batchsize num_hidden
        # state num_layers batchsize num_hidden
        output,state = self.rnn(X)

        return output,state




In [3]:
encoder = Seq2SeqEncoder(vocab_size=10,embed_size=8,num_hiddens=16,num_layers=2)

encoder.eval()
#  四条句子 每个句子有七个单词
X = torch.zeros((4,7),dtype=torch.long)

# 输出形状 7 x 4 x 16  7代表 时间步 也就是七个单词  4代表 四条句子  16代表被提取的维度
output,state = encoder(X)
output.shape

torch.Size([7, 4, 16])

In [4]:
state.shape

torch.Size([2, 4, 16])

In [ ]:
class Seq2SeqDecoder(d2l.Decoder):
    def __init(self,vocab_size,embed_size,num_hiddens,num_layers,dropout=0,**kwargs):
        super(Seq2SeqDecoder,self).__init__(**kwargs)


        # 嵌入
        self.embedding = nn.Embedding(vocab_size,embed_size)

        # RNN  
        self.rnn = nn.GRU(embed_size + num_hiddens,num_hiddens,num_layers,dropout=dropout)

        # 输出层
        self.dense = nn.Linear(num_hiddens,vocab_size)

    # encoder的最后被一个时间步的state 作为Deconde初始化的state
    # 
    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]
    
    def forward(self, X, state):
        X = self.embedding(X).permute(1,0,2)

        # state[-1] 作为最后一层的隐藏状态  形状1 batch_size encoder的 num_hiddens
        # state[-1].repeat(X.shape[0],1,1) 广播操作 变成num_step batch_size num_hiddens
        context = state[-1].repeat(X.shape[0],1,1)

        # 将输入的X 和encoder最后一步的最后一层的隐藏转台拼接在一起  作为decoder的输入
        # 输出形状为(num_steps，batch_size，embed_size + encoder的num_hiddens)
        X_and_context = torch.cat((X,context),2)

        # 输出形状是 num_steps batche_size num_hiddens
        output = self.dense(output).permute(1,0,2)

        return output,state
    

